# Notebook testing
- create cluster redshift 
- test connection redshift
- execute test query

In [1]:
# import library
import configparser
import boto3
import psycopg2
import pandas as pd

## Step 1:
Edit file dwh.cfg \
[AWS] \
KEY=your_key \
SECRET=your_secret

In [2]:
# get config info
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
DWH_CLUSTER_IDENTIFIER = config.get("INFO","DWH_CLUSTER_IDENTIFIER")
DWH_CLUSTER_TYPE       = config.get("INFO","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("INFO","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("INFO","DWH_NODE_TYPE")
DB_NAME                = config.get("CLUSTER","DB_NAME")
DB_PASSWORD            = config.get("CLUSTER","DB_PASSWORD")
DB_USER                = config.get("CLUSTER","DB_USER")
PORT                   =config.get("CLUSTER","DB_PORT")
DWH_IAM_ROLE_NAME      =config.get("INFO","DWH_IAM_ROLE_NAME")

## Create clients IAM,Redshift,EC2

In [3]:
# add role 
iam = boto3.client('iam',
                     region_name='us-east-1',
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)
# create cluter redshift
redshift = boto3.client('redshift',
                     region_name='us-east-1',
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)
# security group
ec2 = boto3.resource('ec2',
                     region_name='us-east-1',
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)

## Step 2: Create IAM role and attach role policy

In [4]:
# attach policy
from botocore.exceptions import ClientError
import json
# Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
    print("1.2 Attaching Policy")
    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']
except Exception as e:
    print(e)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.


## Step 3: Create Redshift cluster

In [5]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
try:
    response = redshift.create_cluster(        
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


In [6]:
# make sure cluster status is available
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.ciuwwfyie3qg.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-01ac5eefec8654329
7,NumberOfNodes,4


## Get endpoint and role arn 
DO NOT RUN THIS unless the cluster status becomes "Available"

In [7]:
# endpoint
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.ciuwwfyie3qg.us-east-1.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::845085505654:role/dwhRole


## Step 4: Open TCP Port

In [8]:
# set ec2 group security
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(PORT),
        ToPort=int(PORT)
    )
except Exception as e:
    print("Error")
    print(e)

ec2.SecurityGroup(id='sg-04af65650c0137411')
Error
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


### Step 5: Connect Redshift and create database, ETL data
Modify HOST = DWH_ENDPOINT print above in dwh.cfg file \
Example: HOST = dwhcluster.ciuwwfyie3qg.us-east-1.redshift.amazonaws.com

In [9]:
# test connection 
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT,*config['CLUSTER'].values()))
#conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT,DB_NAME,DB_USER,DB_PASSWORD,PORT))
cur = conn.cursor()

In [10]:
# drop and create table
%run create_tables.py

Create tables successful !


In [13]:
# etl data (stanging and load to start schema)
%run etl.py

sucess
sucess
Insert successful !


In [15]:
# query example 
# top 10 artist in 2018
query ="""
select a.name, count(a.name) as n_songplays
from songplays_table s
    join artists_table a on s.artist_id = a.artist_id
    join time_table t on t.start_time = s.start_time
where t.year = 2018
group by a.name
order by n_songplays desc
limit 10"""
cur.execute(query)
cur.fetchall()

[('Dwight Yoakam', 37),
 ('Kid Cudi / Kanye West / Common', 10),
 ('Kid Cudi', 10),
 ('Ron Carter', 9),
 ('Lonnie Gordon', 9),
 ('B.o.B', 8),
 ('Usher', 6),
 ('Usher featuring Jermaine Dupri', 6),
 ('Muse', 6),
 ('Richard Hawley And Death Ramps_ Arctic Monkeys', 5)]

In [16]:
# delete cluster
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.ciuwwfyie3qg.us-east-1.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 4, 4, 1, 52, 18, 414000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-07f4c134b6bb11bb2',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-01ac5eefec8654329',
  'AvailabilityZone': 'us-east-1e',
  'PreferredMaintenanceWindow': 'mon:05:30-mon:06:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRoutin